In [32]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import random
import os

## Set random seed for reproducibility

In [33]:
    # 🧠 Set all random seeds for reproducibility
random_state = 42
np.random.seed(random_state)
tf.random.set_seed(random_state)
random.seed(random_state)
os.environ['PYTHONHASHSEED'] = str(random_state)

In [34]:
df = pd.read_csv('../data/dataset0_all_seq.csv')

# df = pd.read_csv('../data/dataset0_mca.csv')

if 'Unnamed: 0' in df.columns:
  df = df.drop(columns='Unnamed: 0')

# Rename ID, POS and SEQ columns

# df = df.rename(columns={
#     'ID': 'transcript_id',
#     'POS': 'transcript_position',
#     'SEQ': '7mer'
# })

In [35]:
df.columns

Index(['transcript_id', 'transcript_position', '7mer', 'label', 'set_type',
       'PreTime_mean', 'PreTime_median', 'PreTime_mode', 'PreTime_std',
       'PreTime_min',
       ...
       'pos6_C', 'pos6_G', 'pos6_T', 'has_GGACT', 'has_GGA', 'has_AGG',
       'has_TGG', 'has_TTA', 'has_GTA', 'has_TAA'],
      dtype='object', length=118)

In [36]:
# Remove 7mer from df

if "7mer" in df.columns:
  df = df.drop(columns='7mer')

## Split df into trainval and test datasets

In [37]:
train_val_df = df[df["set_type"].isin(["Train", "Val"])].copy()
test_df      = df[df["set_type"] == "Test"].copy()

# X_trainval = train_val_df.drop(columns=["gene_id", "transcript_id", "transcript_position", "label", "set_type"])
X_trainval = train_val_df.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
y_trainval = train_val_df["label"]

# X_test = test_df.drop(columns=["gene_id", "transcript_id", "transcript_position", "label", "set_type"])
X_test = test_df.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
y_test = test_df["label"]

In [38]:
train_val_df.head()

,transcript_id,transcript_position,label,set_type,PreTime_mean,PreTime_median,PreTime_mode,PreTime_std,PreTime_min,PreTime_max,...,pos6_C,pos6_G,pos6_T,has_GGACT,has_GGA,has_AGG,has_TGG,has_TTA,has_GTA,has_TAA
0,ENST00000000233,244,0,Train,0.008264,0.00697,0.00398,0.005399,0.00199,0.0339,...,0,0,0,0,0,0,0,0,0,0
1,ENST00000000233,261,0,Train,0.006609,0.00564,0.00498,0.003599,0.00199,0.0222,...,0,1,0,0,0,0,0,0,0,0
2,ENST00000000233,316,0,Train,0.007570,0.00631,0.00498,0.004456,0.00232,0.0299,...,0,1,0,0,0,0,0,0,0,0
3,ENST00000000233,332,0,Train,0.010620,0.00902,0.01020,0.006136,0.00232,0.0370,...,0,0,1,0,0,0,0,0,0,0
4,ENST00000000233,368,0,Train,0.010701,0.00896,0.00398,0.007169,0.00199,0.0478,...,0,0,0,0,1,1,0,0,0,0


### Build model function

For dataset0_all_seq.csv (dataset0 with one-hot-encoded columns):
  - Dense layers of 128 (dropout 0.4, batchnorm), 64 (dropout 0.3, batchnorm), 16, 1
  - All dense layers are relu, final activation layer is sigmoid

For dataset0_mca1.csv (dataset0 with MCA columns):
  - Dense layers of 64 (dropout 0.2, batchnorm), 16 (dropout 0.2, batchnorm), 4 (dropout 0.2, batchnorm), 2, 1
  - All dense layers are relu, final activation layer is sigmoid


--------------------------------------------
- Loss function: binary crossentropy
- Optimizer function: Adam with learning rate of 0.001
- Metrics to evaluate on: roc_auc, pr_auc

In [39]:
def build_model(input_dim):

    tf.random.set_seed(42)
    
    model = Sequential([

        ## This architecture is for MCA
        
        # Dense(64, activation='relu', input_shape=(input_dim,)),
        # BatchNormalization(),
        # Dropout(0.2),

        # Dense(16, activation='relu'),
        # BatchNormalization(),
        # Dropout(0.2),

        # Dense(4, activation='relu'),
        # BatchNormalization(),
        # Dropout(0.2),

        # Dense(2, activation='relu'),

        # Dense(1, activation='sigmoid')

        # # ==========================================================

        ## This architecture is for one hot encoding

        Dense(128, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.4),

        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(16, activation='relu'),

        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
        tf.keras.metrics.AUC(name='roc_auc'),
        tf.keras.metrics.AUC(curve='PR', name='pr_auc')
    ]
    )
    return model

In [40]:
X_trainval.head()

,PreTime_mean,PreTime_median,PreTime_mode,PreTime_std,PreTime_min,PreTime_max,PreTime_p25,PreTime_p75,PreTime_skew,PreTime_kurtosis,...,pos6_C,pos6_G,pos6_T,has_GGACT,has_GGA,has_AGG,has_TGG,has_TTA,has_GTA,has_TAA
0,0.008264,0.00697,0.00398,0.005399,0.00199,0.0339,0.004320,0.01100,1.669388,3.637318,...,0,0,0,0,0,0,0,0,0,0
1,0.006609,0.00564,0.00498,0.003599,0.00199,0.0222,0.004320,0.00797,1.411493,2.225697,...,0,1,0,0,0,0,0,0,0,0
2,0.007570,0.00631,0.00498,0.004456,0.00232,0.0299,0.004400,0.00963,1.597945,3.336704,...,0,1,0,0,0,0,0,0,0,0
3,0.010620,0.00902,0.01020,0.006136,0.00232,0.0370,0.006475,0.01340,1.394256,2.306010,...,0,0,1,0,0,0,0,0,0,0
4,0.010701,0.00896,0.00398,0.007169,0.00199,0.0478,0.005770,0.01390,1.712432,4.387780,...,0,0,0,0,1,1,0,0,0,0


### K-fold cross validation

- Perform K-fold cross validation with n_splits=5
- Train and validation set is split using StratifiedKFold
- Standard scaler fitted on X_train, then transformed on X_train and X_val at each fold
- Compute class weights using X_train
- Early stopping is monitored val_pr_auc, if it does not improve within 8 epochs then perform early stopping
- Evaluate roc_auc and pr_auc of each fold, and at the end the average roc_auc and pr_auc of the 5 folds are taken alongside the standard deviation

In [41]:
K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

cv_auc = []
cv_pr_auc = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_trainval, y_trainval), 1):
    print(f"\n===== Fold {fold}/{K} =====")

    X_train, X_val = X_trainval.iloc[train_idx], X_trainval.iloc[val_idx]
    y_train, y_val = y_trainval.iloc[train_idx], y_trainval.iloc[val_idx]

    # Perform scaling at each fold fitting only on training set
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Compute class weights for imbalance
    classes = np.unique(y_train)
    cw = compute_class_weight('balanced', classes=classes, y=y_train)
    class_weights = dict(zip(classes, cw))


    # Adjust ratio to be *less extreme* than fully balanced (e.g., 70–80% of computed weight)
    # class_weights[1] = class_weights[1] * 0.5  # downscale minority class weight slightly
    # class_weights[0] = class_weights[0] * 1.5  # upweight majority class mildly

    model = build_model(X_train.shape[1])
    early_stop = EarlyStopping(monitor='val_pr_auc', mode='max', patience=8, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        class_weight=class_weights,
        callbacks=[early_stop],
        verbose=0
    )

    y_val_prob = model.predict(X_val)
    roc_auc = roc_auc_score(y_val, y_val_prob)
    precision, recall, _ = precision_recall_curve(y_val, y_val_prob)
    pr_auc = auc(recall, precision)

    print(f"Fold {fold}: ROC-AUC={roc_auc:.4f}, PR-AUC={pr_auc:.4f}")
    cv_auc.append(roc_auc)
    cv_pr_auc.append(pr_auc)

print("\n===== Cross-Validation Summary =====")
print(f"Mean ROC-AUC: {np.mean(cv_auc):.4f} ± {np.std(cv_auc):.4f}")
print(f"Mean PR-AUC : {np.mean(cv_pr_auc):.4f} ± {np.std(cv_pr_auc):.4f}")


===== Fold 1/5 =====
680/680 [==============================] - 2s 3ms/step
Fold 1: ROC-AUC=0.9072, PR-AUC=0.3923

===== Fold 2/5 =====
680/680 [==============================] - 2s 3ms/step
Fold 2: ROC-AUC=0.9091, PR-AUC=0.4203

===== Fold 3/5 =====
680/680 [==============================] - 2s 2ms/step
Fold 3: ROC-AUC=0.9089, PR-AUC=0.4131

===== Fold 4/5 =====
680/680 [==============================] - 2s 2ms/step
Fold 4: ROC-AUC=0.9179, PR-AUC=0.4637

===== Fold 5/5 =====
680/680 [==============================] - 1s 2ms/step
Fold 5: ROC-AUC=0.9081, PR-AUC=0.4172

===== Cross-Validation Summary =====
Mean ROC-AUC: 0.9102 ± 0.0039
Mean PR-AUC : 0.4213 ± 0.0233


## Train on entire training set, validate on validation set then evaluate on test set

- Fit standard scaler on X_train, then transform X_val and X_test
- Computes class weights from y_train
- Validate during training using validation set
- Evaluate fitted model on X_test_scaled
- Calculate roc_auc_score and pr_auc_score

In [42]:
train_df = df[df["set_type"] == "Train"].copy()
val_df = df[df["set_type"] == "Train"].copy()
test_df = df[df["set_type"] == "Test"].copy()

X_train = train_df.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
y_train = train_df["label"]

X_val = val_df.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
y_val = val_df["label"]

X_test = test_df.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
y_test = test_df["label"]

In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [44]:
classes = np.unique(y_train)
cw = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, cw))

# Adjust ratio to be *less extreme* than fully balanced (e.g., 70–80% of computed weight)
# class_weights[1] = class_weights[1] * 0.5  # downscale minority class weight slightly
# class_weights[0] = class_weights[0] * 1.5  # upweight majority class mildly

early_stop = EarlyStopping(monitor='val_pr_auc', mode='max', patience=10, restore_best_weights=True)

final_model = build_model(X_train_scaled.shape[1])
final_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    class_weight=class_weights,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stop],
    verbose=1
)

y_pred_prob = final_model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)

print(classification_report(y_test, y_pred, digits=4))
print(f"Test ROC-AUC: {roc_auc:.4f}")
print(f"Test PR-AUC : {pr_auc:.4f}")

Epoch 1/100
2995/2995 [==============================] - 22s 7ms/step - loss: 0.4844 - roc_auc: 0.8517 - pr_auc: 0.2780 - val_loss: 0.4314 - val_roc_auc: 0.9165 - val_pr_auc: 0.4233
Epoch 2/100
2995/2995 [==============================] - 21s 7ms/step - loss: 0.4106 - roc_auc: 0.8949 - pr_auc: 0.3592 - val_loss: 0.4077 - val_roc_auc: 0.9203 - val_pr_auc: 0.4292
Epoch 3/100
2995/2995 [==============================] - 20s 7ms/step - loss: 0.3915 - roc_auc: 0.9050 - pr_auc: 0.3872 - val_loss: 0.3920 - val_roc_auc: 0.9263 - val_pr_auc: 0.4642
Epoch 4/100
2995/2995 [==============================] - 20s 7ms/step - loss: 0.3804 - roc_auc: 0.9103 - pr_auc: 0.4064 - val_loss: 0.3416 - val_roc_auc: 0.9315 - val_pr_auc: 0.4730
Epoch 5/100
2995/2995 [==============================] - 20s 7ms/step - loss: 0.3711 - roc_auc: 0.9146 - pr_auc: 0.4072 - val_loss: 0.3788 - val_roc_auc: 0.9287 - val_pr_auc: 0.4758
Epoch 6/100
2995/2995 [==============================] - 17s 6ms/step - loss: 0.3674 - roc

In [45]:
print(class_weights)

{0: 0.5239571373899732, 1: 10.935303514376997}


## Train on full dataset for leaderboard output

In [46]:
# Full model training

df_new = df.rename(columns={
    'ID': 'transcript_id',
    'POS': 'transcript_position',
    'SEQ': '7mer'
})

X_0 = df_new.drop(columns=["transcript_id", "transcript_position", "label", "set_type"])
X_settype = df_new["set_type"]
y_0 = df_new['label']

scaler_final = StandardScaler()
X_0_scaled = scaler_final.fit_transform(X_0)

classes = np.unique(y_0)
cw = compute_class_weight('balanced', classes=classes, y=y_0)
class_weights = dict(zip(classes, cw))

early_stop = EarlyStopping(monitor='val_pr_auc', mode='max', patience=10, restore_best_weights=True)

final_model = build_model(X_0_scaled.shape[1])
final_model.fit(
    X_0_scaled, y_0,
    epochs=100,
    batch_size=32,
    class_weight=class_weights,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

y_0_prob = final_model.predict(X_0_scaled)

Epoch 1/100
3046/3046 [==============================] - 20s 6ms/step - loss: 0.4780 - roc_auc: 0.8540 - pr_auc: 0.2536 - val_loss: 0.4105 - val_roc_auc: 0.9048 - val_pr_auc: 0.4044
Epoch 2/100
3046/3046 [==============================] - 15s 5ms/step - loss: 0.4105 - roc_auc: 0.8947 - pr_auc: 0.3579 - val_loss: 0.3905 - val_roc_auc: 0.9120 - val_pr_auc: 0.4300
Epoch 3/100
3046/3046 [==============================] - 14s 4ms/step - loss: 0.3878 - roc_auc: 0.9064 - pr_auc: 0.3932 - val_loss: 0.4167 - val_roc_auc: 0.9145 - val_pr_auc: 0.4453
Epoch 4/100
3046/3046 [==============================] - 15s 5ms/step - loss: 0.3799 - roc_auc: 0.9102 - pr_auc: 0.4063 - val_loss: 0.3731 - val_roc_auc: 0.9193 - val_pr_auc: 0.4564
Epoch 5/100
3046/3046 [==============================] - 15s 5ms/step - loss: 0.3683 - roc_auc: 0.9159 - pr_auc: 0.4198 - val_loss: 0.3202 - val_roc_auc: 0.9180 - val_pr_auc: 0.4612
Epoch 6/100
3046/3046 [==============================] - 13s 4ms/step - loss: 0.3595 - roc

### Save the final model

In [ ]:
# final_model.save("../models/site_level_model_ohe1.keras")
# final_model.save("../models/site_level_model_mca1.keras")

### Checking predictions of final model on dataset0

In [47]:
df_transcript = df_new[["transcript_id", "transcript_position"]]

df_transcript['score'] = y_0_prob

df_transcript.head()

C:\Users\nicho\AppData\Local\Temp\ipykernel_22240\1062056413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transcript['score'] = y_0_prob


,transcript_id,transcript_position,score
0,ENST00000000233,244,0.099253
1,ENST00000000233,261,0.266505
2,ENST00000000233,316,0.025806
3,ENST00000000233,332,0.374135
4,ENST00000000233,368,0.053117


### Save the dataset0 predictions to csv

In [ ]:
# df_transcript.to_csv(f"../intermediate_submissions/genetherapy_dataset0_2.csv", index=False)

### Evaluate dataset0 results

In [48]:
roc_auc = roc_auc_score(y_0, y_0_prob)
precision, recall, _ = precision_recall_curve(y_0, y_0_prob)
pr_auc = auc(recall, precision)

print("Dataset 0 evaluation")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"PR-AUC : {pr_auc:.4f}")

Dataset 0 evaluation
ROC-AUC: 0.9479
PR-AUC : 0.5334
